In [ ]:
# Preperare distributed
using Distributed
addprocs(Sys.CPU_THREADS - nprocs() - 1)
print("Number of workers: ", nprocs(), "\nNumber of CPU threads: ", Sys.CPU_THREADS, "\n")

using JLD2
@everywhere using SpecialFunctions
using Printf


@everywhere using LoopVectorization
using BenchmarkTools

@everywhere using Plots
include("distributed_gif.jl")

using Base.Threads
print("Number of threads: ", Threads.nthreads(), "\n") # Check number of threads available

# Problem 1.2

In [ ]:
N = 100
L = 1.0
dx= L/N
D = 1.0
dt = 0.00001
c_0 = zeros(N, N)
c_0[:,end] .= 1.0

In [ ]:
# Plotting parameters
ticks = (1:N/4:N+1, 0:L/4:L)
lims = (1, N+1)
heatmap_kwargs = Dict(
    :aspect_ratio => 1,
    :xlabel => "x", 
    :ylabel => "y", 
    :xticks => ticks, 
    :yticks => ticks, 
    :xlims => lims, 
    :ylims => lims, 
    :dpi => 150
)

heatmap_kwargs
heatmap(c_0', title="Initial condition"; heatmap_kwargs...)

## 1.2D

In [ ]:
"Enable benchmarks to test which version is faster. Set `do_bench = true` to run benchmarks."
do_bench = false

In [ ]:
function c_next(c::Matrix{Float64}, D::Float64, dx::Float64, dt::Float64)
    N = size(c, 1)
    c_new = similar(c)

    # Apply boundary conditions
    c_new[:, 1] .= 0.0  # Bottom boundary
    c_new[:, end] .= 1.0  # Top boundary

    Fo = D * dt / dx^2

    @inbounds @turbo for i in 2:N-1, j in 2:N-1
        c_new[i, j] = c[i, j] + Fo * (c[i+1, j] + c[i-1, j] + c[i, j+1] + c[i, j-1] - 4 * c[i, j])
    end

    # Periodic in x-direction
    @inbounds for j in 2:N-1
        c_new[1, j] = c[1, j] + Fo * (c[2, j] + c[N, j] + c[1, j+1] + c[1, j-1] - 4 * c[1, j])
        c_new[N, j] = c[N, j] + Fo * (c[1, j] + c[N-1, j] + c[N, j+1] + c[N, j-1] - 4 * c[N, j])
    end

    return c_new
end
if do_bench
    @benchmark c_next($c_0, $D, $dx, $dt)
end

In [ ]:
function c_next_single_loop(c::Matrix{Float64}, D::Float64, dx::Float64, dt::Float64)
    N = size(c, 1)
    c_new = zeros(N, N)
    @inbounds @turbo for i in 1:N, j in 2:N-1
        # Project x-end to x-start (periodic boundary condition)
        i_right = i == N ? 2 : i + 1
        i_left = i == 1 ? N - 1 : i - 1

        c_new[i, j] = c[i, j] + (D * dt / dx^2) * (c[i_right, j] + c[i_left, j] + c[i, j+1] + c[i, j-1] - 4 * c[i, j])
    end

    # Apply boundary conditions
    c_new[:, 1] .= 0.0  # Left boundary
    c_new[:, end] .= 1.0  # Right boundary
    return c_new
end
if do_bench
    @benchmark c_next_single_loop($c_0, $D, $dx, $dt)
end

In [ ]:
function c_next_dist_turbo(c::Matrix{Float64}, D::Float64, dx::Float64, dt::Float64)
    N = size(c, 1)
    c_new = similar(c)

    # Apply boundary conditions
    c_new[:, 1] .= 0.0  # Left boundary
    c_new[:, end] .= 1.0  # Right boundary

    Fo = D * dt / dx^2

    @inbounds @distributed for i in 2:N-1
        @inbounds @turbo for j in 2:N-1
            c_new[i, j] = c[i, j] + Fo * (c[i+1, j] + c[i-1, j] + c[i, j+1] + c[i, j-1] - 4 * c[i, j])
        end
    end

    @inbounds @turbo for j in 2:N-1
        c_new[1, j] = c[1, j] + Fo * (c[2, j] + c[N, j] + c[1, j+1] + c[1, j-1] - 4 * c[1, j])
        c_new[N, j] = c[N, j] + Fo * (c[1, j] + c[N-1, j] + c[N, j+1] + c[N, j-1] - 4 * c[N, j])
    end

    return c_new
end
if do_bench
    @benchmark c_next_dist_turbo($c_0, $D, $dx, $dt)
end

In [ ]:
function c_next_dist(c::Matrix{Float64}, D::Float64, dx::Float64, dt::Float64)
    N = size(c, 1)
    c_new = similar(c)

    # Apply boundary conditions
    c_new[:, 1] .= 0.0  # Bottom boundary
    c_new[:, end] .= 1.0  # Top boundary

    Fo = D * dt / dx^2

    @inbounds @distributed for z in 1:(N-2)*(N-2)
        i = mod(div(z-1, N-2), N-2) + 2
        j = mod(z-1, N-2) + 2
        c_new[i, j] = c[i, j] + Fo * (c[i+1, j] + c[i-1, j] + c[i, j+1] + c[i, j-1] - 4 * c[i, j])
    end

    @inbounds @turbo for j in 2:N-1
        c_new[1, j] = c[1, j] + Fo * (c[2, j] + c[N, j] + c[1, j+1] + c[1, j-1] - 4 * c[1, j])
        c_new[N, j] = c[N, j] + Fo * (c[1, j] + c[N-1, j] + c[N, j+1] + c[N, j-1] - 4 * c[N, j])
    end

    return c_new
end
if do_bench
    @benchmark c_next_dist($c_0, $D, $dx, $dt)
end

In [ ]:
function propagate_c_diffusion(c::Matrix, L::Float64, N::Int, D::Float64, t_0::Float64, t_f::Float64, dt::Float64)
    c_curr = copy(c)
    dx = L / N

    # Check stability condition
    if D * dt / dx^2 > 0.25
        error("Stability condition violated: D*dt/dx^2 must be <= 0.25")
    end

    for t in t_0:dt:t_f
        c_curr = c_next(c_curr, D, dx, dt)
    end

    return c_curr
end

## 1.2E
Compare with analytical solution.

In [ ]:
do_cache = true

In [ ]:
if do_cache && isfile("diffusion_results.jld2")
    @load "diffusion_results.jld2" results
    c_0001, c_001, c_01, c_1 = results
else
    c_0001 = propagate_c_diffusion(c_0, L, N, D, 0.0, 0.001, dt)
    c_001 = propagate_c_diffusion(c_0001, L, N, D, 0.001, 0.01, dt)
    c_01 = propagate_c_diffusion(c_001, L, N, D, 0.01, 0.1, dt)
    c_1 = propagate_c_diffusion(c_01, L, N, D, 0.1, 1.0, dt)

    results = [c_0001, c_001, c_01, c_1]

    if do_cache
        @save "diffusion_results.jld2" results
    end
end


p_yprofile = plot(c_0001[1,:], title="y-profile", xlabel="x", ylabel="y", label = "t = 0.001 (numerical)", dpi=150)
plot!(p_yprofile, c_001[1,:], label = "t = 0.01 (numerical)")
plot!(p_yprofile, c_01[1,:], label = "t = 0.1 (numerical)")
plot!(p_yprofile, c_1[1,:], label = "t = 1.0 (numerical)")
display(p_yprofile)

In [ ]:
function c_anal(x::Float64, t::Float64; D::Float64 = D, i_max::Int = 100)
    denom = 2 * sqrt(D * t)

    function c_anal_i(x::Float64, t::Float64, i::Int, D::Float64)
        return erfc((1 - x + 2i) / denom) - erfc((1 + x + 2i) / denom)
    end
    
    if i_max > 50_000
        return @distributed (+) for i in 0:i_max
            c_anal_i(x, t, i, D)
        end
    else
        return sum(c_anal_i(x, t, i, D) for i in 0:i_max)
    end
end

analytical_sol = (t) -> [c_anal(x, t) for x in LinRange(0, L, N)]

In [ ]:
plot_anal = plot(p_yprofile, analytical_sol(0.001), linestyle = :dot, color=:red, label = "t = 0.001 (analytical)", dpi=150)
plot!(plot_anal, analytical_sol(0.01), linestyle=:dot, color=:blue, label="t = 0.01 (analytical)")
plot!(plot_anal, analytical_sol(0.1), linestyle = :dot, color=:orange, label = "t = 0.1 (analytical)")
plot!(plot_anal, analytical_sol(1.0), linestyle=:dot, color=:yellow, label="t = 1.0 (analytical)")

## 1.2F

In [ ]:
# Plot heatmaps at these time points
plot(
    heatmap(c_0', title="t = 0"; heatmap_kwargs...), 
    heatmap(c_0001', title="t = 0.001"; heatmap_kwargs...), 
    heatmap(c_001', title="t = 0.01"; heatmap_kwargs...), 
    heatmap(c_01', title="t = 0.1"; heatmap_kwargs...), 
    heatmap(c_1', title="t = 1.0"; heatmap_kwargs...),
    size=(1200,800),
)

## 1.2 G

In [ ]:
include("distributed_gif.jl")

plots = (() -> begin
    frames::Vector{Plots.Plot{Plots.GRBackend}} = []
    c_i = c_0
    for i in 0:317
        t_prev = (i-1)^2 * 0.00001
        t = (i)^2 * 0.00001
        if t >= 1.0
            break
        end

        c_i = propagate_c_diffusion(c_i, L, N, D, t_prev, t, dt)
        push!(frames, plot(heatmap(c_i', title=@sprintf("t = %0.3f", t); heatmap_kwargs...)))
    end
    frames
end)()
anim = Animation()
for p in plots
    frame(anim, p)
end
gif(anim, "diffusion_evolution.gif", fps=30)
# distributed_gif(plots, "diffusion_evolution.gif", fps=30, do_palette=true, width=600)